In [ ]:
import json
import urllib.request
import re
import pandas as pd

In [ ]:
def get_json(title):
  baseurl = "https://marvel.fandom.com/api.php?"
  action = "action=query"
  title = "titles={}".format(title.replace(" ", "_"))
  content = "prop=revisions&rvprop=content&rvslots=*"
  dataformat ="format=json"

  query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    
  wikiresponse = urllib.request.urlopen(query)
  wikidata = wikiresponse.read()
  wikitext = wikidata.decode('utf-8')


  return json.loads(wikitext)

def displayWiki(wiki):
    code = str(list(wiki["query"]["pages"].keys())[0])
    title = wiki["query"]["pages"][code]["title"]
    content = wiki["query"]["pages"][code]["revisions"][0]["slots"]["main"]["*"]
    return title, content

In [ ]:
teams = None
with open("teams.txt", "r") as f:
  teams = f.read().split("\n")[:-1]
teams

In [ ]:
regex_links = r"\[\[(.*?)(?:|\|.*?)\]\]"

In [ ]:
def createDataFrame(teams):
  df = pd.DataFrame(columns=["team_name", "leaders", "current_members", "former_members", "allies", "enemies"])
  
  for team in teams:
    content = displayWiki(get_json(team))[1]
    header = re.split(r"\| First", content)[0]

    name = re.sub(" \(Earth-.*", "", team)
    
    leaders_raw = re.findall(r"Leaders.*?\| CurrentMembers", header, flags=re.DOTALL)[0]
    leaders = re.findall(regex_links, leaders_raw)

    current_member_raw = re.findall(r"\| CurrentMembers.*?\| FormerMembers", header, flags=re.DOTALL)[0]
    current_member = re.findall(regex_links, current_member_raw)

    former_member_raw = re.findall(r"\| FormerMembers.*?\| Allies", header, flags=re.DOTALL)[0]
    former_member = re.findall(regex_links, former_member_raw)

    allies_raw = re.findall(r"\| Allies.*?\| Enemies", header, flags=re.DOTALL)[0]
    allies = re.findall(regex_links, allies_raw)

    enemies_raw = re.findall(r"\| Enemies.*?\| Origin", header, flags=re.DOTALL)[0]
    enemies = re.findall(regex_links, enemies_raw)

    row = {
      "team_name"      : name,
      "leaders"        : leaders,
      "current_members": current_member,
      "former_members" : former_member,
      "allies"         : allies,
      "enemies"        : enemies
    }
    
    df = df.append(row, ignore_index=True)
  
  return df

In [ ]:
marvel_df = createDataFrame(teams)
marvel_df.head()

In [ ]:
marvel_df.to_csv("marvel_teams.csv")